<a href="https://colab.research.google.com/github/matheusfbonfim/CNN-Face-Mask-Detection/blob/main/CNN_Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>